In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from  scipy.stats import norm,skew
import time

In [2]:
train=pd.read_csv("data/train.csv",parse_dates=[2], low_memory=False)
test=pd.read_csv("data/test.csv",parse_dates=[3],low_memory=False)
store=pd.read_csv("data/store.csv")


In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb

In [4]:
print(train["Date"].min(),train["Date"].max())
train.sort_values(["Date"],inplace=True, kind="mergesort")
train.reset_index(drop=True,inplace=True)

2013-01-01 00:00:00 2015-07-31 00:00:00


In [5]:
test["Open"]=test["Open"].fillna(1)

In [6]:
store.isnull().sum()/store.shape[0]

Store                        0.000000
StoreType                    0.000000
Assortment                   0.000000
CompetitionDistance          0.002691
CompetitionOpenSinceMonth    0.317489
CompetitionOpenSinceYear     0.317489
Promo2                       0.000000
Promo2SinceWeek              0.487892
Promo2SinceYear              0.487892
PromoInterval                0.487892
dtype: float64

In [7]:
store.fillna(0,inplace=True)

In [8]:
train=pd.merge(train,store,on="Store",how="left")
test=pd.merge(test,store,on="Store",how="left")

In [9]:
for df in [train,test]:
    df["year"]=df.Date.dt.year
    df["month"]=df.Date.dt.month
    df["day"]=df.Date.dt.day
    df["day"]=df.Date.dt.day
    assert np.all(df.DayOfWeek-1==df["Date"].dt.dayofweek)
    df["dayofyear"]=df.Date.dt.dayofyear
    df["weekofyear"]=df.Date.dt.weekofyear
    df.drop("Date",axis=1,inplace=True)
    

In [10]:
for df in [train,test]:
    df["CompetitionOpen"]=((df["year"]-df["CompetitionOpenSinceYear"])*12 
                          + (df["month"]-df["CompetitionOpenSinceMonth"]))
    df["CompetitionOpen"]=df["CompetitionOpen"].apply(lambda x : x if x>0 else 0)
    df["PromoOpen"]=((df["year"]-df["Promo2SinceYear"]) * 12
                     + (df["weekofyear"]-df["Promo2SinceWeek"])/4)
    df["PromoOpen"]=df["PromoOpen"].apply(lambda x : x if x>0 else 0)
    

In [11]:
month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',
             7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
def check(row):
    if isinstance(row["PromoInterval"],str) and month2str[row["month"]] in row["PromoInterval"]:
        if (row["year"] > row["Promo2SinceYear"] or
            (row['year'] == row['Promo2SinceYear'] and row['weekofyear'] > row['Promo2SinceWeek'])):
            return 1
    return 0

for df in [train,test]:
    df["IsPromoMonth"]=df.apply(lambda row:check(row), axis=1)

### train은 Open 평균이 의미가 있으나, test는 Open 평균이 모두 같으므로 train의 opne 평균을 test에도 적용한다

In [12]:
# train
groups=train[["Store","Open"]].groupby("Store").mean()
groups.rename(columns={"Open":"shopavgopen"},inplace=True)
train = pd.merge(train, groups, how="left", on="Store")
test = pd.merge(test, groups, how="left", on="Store")

In [13]:
groups=train[["Store","Sales","Customers"]].groupby("Store").sum()
groups["ShopAvgSalePerCustomer"]=groups["Sales"]/groups["Customers"]
del groups["Sales"],groups["Customers"]
train = pd.merge(train, groups, how="left", on="Store")
test = pd.merge(test, groups, how="left", on="Store")

In [14]:
groups=train[["Store","SchoolHoliday"]].groupby("Store").mean()
groups.columns=["ShopAvgSchoolHoliday"]
train = pd.merge(train, groups, how="left", on="Store")
test = pd.merge(test, groups, how="left", on="Store")

In [15]:
train.columns

Index(['Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval', 'year', 'month', 'day', 'dayofyear',
       'weekofyear', 'CompetitionOpen', 'PromoOpen', 'IsPromoMonth',
       'shopavgopen', 'ShopAvgSalePerCustomer', 'ShopAvgSchoolHoliday'],
      dtype='object')

In [16]:
groups1=train[["Store","Sales"]].groupby("Store").sum()
groups2=train[train["StateHoliday"]!=0][["Store","Sales"]].groupby("Store").sum()
groups=pd.merge(groups1,groups2,on="Store")
groups["ShopSalesHoliday"]=groups["Sales_y"]/groups["Sales_x"]
del groups["Sales_x"],groups["Sales_y"]
train = pd.merge(train, groups, how="left", on="Store")
test = pd.merge(test, groups, how="left", on="Store")

In [17]:
groups1=train[["Store","Sales"]].groupby("Store").sum()
groups2=train[train["IsPromoMonth"]==1][["Store","Sales"]].groupby("Store").sum()
groups=pd.merge(groups1,groups2,on="Store")
groups["ShopSalesPromo"]=groups["Sales_y"]/groups["Sales_x"]
del groups["Sales_x"],groups["Sales_y"]
train = pd.merge(train, groups, how="left", on="Store")
test = pd.merge(test, groups, how="left", on="Store")

In [18]:
groups1=train[["Store","Sales"]].groupby("Store").sum()
groups2=train[train["DayOfWeek"]==6][["Store","Sales"]].groupby("Store").sum()
groups=pd.merge(groups1,groups2,on="Store")
groups["ShopSalesSaturday"]=groups["Sales_y"]/groups["Sales_x"]
del groups["Sales_x"],groups["Sales_y"]
train = pd.merge(train, groups, how="left", on="Store")
test = pd.merge(test, groups, how="left", on="Store")

In [19]:
assert np.all(train[train.Open==0]["Sales"]==0)
train=train[train["Sales"]!=0]
del train["Open"]
test_close_ind=np.where(test["Open"]==0)[0]
del test["Open"]

In [20]:
for col in ["StateHoliday", "StoreType", "Assortment", "DayOfWeek", "month", "PromoInterval"]:
    for val in train[col].unique():
        new_col_name=col+"_"+str(val)
        train[new_col_name]=(train[col]==val).astype(int)
        test[new_col_name]=(test[col]==val).astype(int)
del train["PromoInterval"],test["PromoInterval"]

In [21]:
for col in ["StateHoliday","StoreType","Assortment"]:
    le=LabelEncoder()
    train[col]=le.fit_transform(train[col])
    test[col]=le.transform(test[col])

In [22]:
train.StateHoliday.unique(),train.StoreType.unique(),train.Assortment.unique()

(array([1, 0, 2, 3], dtype=int64),
 array([1, 0, 2, 3], dtype=int64),
 array([0, 1, 2], dtype=int64))

In [23]:
train.StateHoliday.unique(),train.StoreType.unique(),train.Assortment.unique()

(array([1, 0, 2, 3], dtype=int64),
 array([1, 0, 2, 3], dtype=int64),
 array([0, 1, 2], dtype=int64))

In [24]:
y_train=np.array(train.Sales)
#y_train=train.Sales
train.drop("Sales",axis=1,inplace=True)
train.drop("Customers",axis=1,inplace=True)
test_id=test["Id"]
test.drop("Id",axis=1,inplace=True)

In [25]:
y_train=np.log1p(y_train)

In [37]:
def rmspe(y_pred,y_true):
    y_pred=y_pred[y_pred!=0]
    y_true=y_true[y_true!=0]
    err=np.sqrt(np.mean((1-y_pred/y_true)**2))
    return err
def rmspe_xgb(y_pred,y_true):
#    y_true=y_true.get_label()
    err=rmspe(np.expm1(y_pred),np.expm1(y_true))
    return "rmspe",err

In [27]:
# start=time.time()
# valid_mask=(train.year==2015) & (train.dayofyear>=171)
# train1,y_train1=train[~valid_mask],y_train[~valid_mask]
# train2,y_train2=train[valid_mask],y_train[valid_mask]
# reg=xgb.XGBRegressor(n_estimators=5000,objective="reg:squarederror",max_depth=10,
#                     learning_reate=0.03,colsample_bytree=0.7,subsample=0.9,
#                     random_state=0, tree_method="gpu_hist")
# reg.fit(train1,y_train1,eval_set=[(train1,y_train1),(train2,y_train2)],
#        eval_metric=rmspe_xgb,early_stopping_rounds=100,verbose=100)
# best_iteration=reg.best_iteration
# print("ellapse",time.time()-start)

In [28]:
# pred=reg.predict(test)
# train_pre=reg.predict(train)
# print(rmspe(np.expm1(train_pre),np.expm1(y_train)))

In [29]:
# pred=np.expm1(reg.predict(test))
# pred[test_close_ind]=0
# submission=pd.DataFrame({"Id":test_id,"Sales":pred},columns=["Id","Sales"])

In [30]:
from datetime import datetime
now=datetime.now()
#submission.to_csv("data/{0:02d}{1:02d}{2:02d}{3:02d}_xgb_submission.csv".format(now.year,now.month,now.day,now.hour),index=False)

In [31]:
474/3303
0.11943

0.11943

## lgb parameter tuning

In [41]:
train.rename(columns = {"PromoInterval_Jan,Apr,Jul,Oct": "PromoInterval_Jan_Apr_Jul_Oct"}, inplace = True)
train.rename(columns = {"PromoInterval_Feb,May,Aug,Nov": "PromoInterval_Feb_May_Aug_Nov"}, inplace = True)
train.rename(columns = {"PromoInterval_Mar,Jun,Sept,Dec": "PromoInterval_Mar_Jun_Sept_Dec"}, inplace = True)

test.rename(columns = {"PromoInterval_Jan,Apr,Jul,Oct": "PromoInterval_Jan_Apr_Jul_Oct"}, inplace = True)
test.rename(columns = {"PromoInterval_Feb,May,Aug,Nov": "PromoInterval_Feb_May_Aug_Nov"}, inplace = True)
test.rename(columns = {"PromoInterval_Mar,Jun,Sept,Dec": "PromoInterval_Mar_Jun_Sept_Dec"}, inplace = True)

In [33]:
from sklearn.model_selection import KFold
import lightgbm as lgbm

learning_rate=0.1
num_leaves=15

params = {"boosting_type": "gbdt",
          "learning_rate": learning_rate,
          "num_leaves": num_leaves,
          "n_estimators":5000,
          "early_stopping_rounds":100
          }

In [46]:
def rmspe(y_pred,y_true):
    y_pred=y_pred[y_pred!=0]
    y_true=y_true[y_true!=0]
    err=np.sqrt(np.mean((1-y_pred/y_true)**2))
    return err
def rmspe_xgb(y_pred,y_true):
    y_true=y_true.get_label()
    err=rmspe(np.expm1(y_pred),np.expm1(y_true))
    return "rmspe",err
def rmspe_lgb(y_pred,y_true):
    pre=np.expm1(y_pred)
    tru=np.expm1(y_true)
    err=np.sqrt(np.mean((1-pre/tru)**2))
    return "rmspe",err,True

In [40]:
NFOLDS=5
kfold=KFold(n_splits=NFOLDS, shuffle=True, random_state=218)

best_trees = []
fold_scores = []
cv_train = np.zeros(len(y_train))
cv_pred = np.zeros(len(test_id))

for i,(idx_trn,idx_val) in enumerate(kfold.split(train,y_train)):
    print(len(idx_trn)+len(idx_val))
    x_trn,y_trn=train.iloc[idx_trn],y_train[idx_trn]
    x_val,y_val=train.iloc[idx_val],y_train[idx_val]
    
    reg=lgbm.LGBMRegressor(
        n_estimators=5000,
        #learning_rate=0.1,
        #num_leaves=123,
        #colsample_bytree=.8,
        #subsample=.9,
        #max_depth=15,
        #reg_alpha=.1,
        #reg_lambda=.1,
        #min_split_gain=.01,
        #min_child_weight=2
        verbose_eval=100,
        early_stopping_rounds=100
    )
    reg.fit(x_trn,y_trn,
           eval_set=[(x_trn,y_trn),(x_val,y_val)],
           eval_metric=rmspe_xgb)
    best_trees.append(reg.best_iteration_)
    cv_pred += reg.predict(test)
    cv_train[idx_val] += reg.predict(x_val)

cv_pred /= NFOLDS
print("score",rmspe_lgb(cv_train,y_train))

844338


c:\python\python36\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


AttributeError: 'numpy.ndarray' object has no attribute 'get_label'

In [ ]:
cv_pred[test_close_ind]=0
submission=pd.DataFrame({"Id":test_id,"Sales":np.round(np.expm1(cv_pred),3)},columns=["Id","Sales"])

In [ ]:
from datetime import datetime
now=datetime.now()
submission.to_csv("data/{0:02d}{1:02d}{2:02d}{3:02d}_lgbm_submission.csv".format(now.year,now.month,now.day,now.hour),index=False)

## 3% 버전 (no validation)

In [47]:
gbm = lgb.LGBMRegressor(n_estimators=5000)
gbm.fit(train, y_train,
        eval_metric=rmspe_lgb)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=5000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [51]:
def rmspe_lgb(y_pred,y_true):
    pre=np.expm1(y_pred)
    tru=np.expm1(y_true)
    err=np.sqrt(np.mean((1-pre/tru)**2))
    return "rmspe",err,False

In [53]:
NFOLDS=5
kfold=KFold(n_splits=NFOLDS, shuffle=True, random_state=218)

best_trees = []
fold_scores = []
cv_train = np.zeros(len(y_train))
pred = np.zeros(len(test_id))

for i,(idx_trn,idx_val) in enumerate(kfold.split(train,y_train)):
    x_trn,y_trn=train.iloc[idx_trn],y_train[idx_trn]
    x_val,y_val=train.iloc[idx_val],y_train[idx_val]
    
    reg=lgbm.LGBMRegressor(n_estimators=1000,verbose=250, early_stopping_rounds=150)
    reg.fit(x_trn,y_trn,
           eval_set=[(x_trn,y_trn),(x_val,y_val)],
           eval_metric=rmspe_lgb)
    pred += reg.predict(test,num_iteration=reg.best_iteration_)
    cv_train[idx_val] += reg.predict(x_val,num_iteration=reg.best_iteration_)
    del reg, x_trn, y_trn, x_val, y_val

pred /= NFOLDS
print("score",rmspe_lgb(cv_train,y_train))

c:\python\python36\lib\site-packages\lightgbm\engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	training's l2: 0.167706	training's rmspe: 0.475899	valid_1's l2: 0.16881	valid_1's rmspe: 0.478313
Training until validation scores don't improve for 150 rounds
[2]	training's l2: 0.156925	training's rmspe: 0.458268	valid_1's l2: 0.157967	valid_1's rmspe: 0.460625
[3]	training's l2: 0.147939	training's rmspe: 0.442622	valid_1's l2: 0.14893	valid_1's rmspe: 0.445002
[4]	training's l2: 0.140379	training's rmspe: 0.429192	valid_1's l2: 0.14134	valid_1's rmspe: 0.431656
[5]	training's l2: 0.134024	training's rmspe: 0.41791	valid_1's l2: 0.134928	valid_1's rmspe: 0.420298
[6]	training's l2: 0.128595	training's rmspe: 0.407654	valid_1's l2: 0.129425	valid_1's rmspe: 0.409865
[7]	training's l2: 0.124051	training's rmspe: 0.399541	valid_1's l2: 0.124859	valid_1's rmspe: 0.401711
[8]	training's l2: 0.119918	training's rmspe: 0.392022	valid_1's l2: 0.120703	valid_1's rmspe: 0.394241
[9]	training's l2: 0.116423	training's rmspe: 0.385354	valid_1's l2: 0.117202	valid_1's rmspe: 0.38757
[10]	tr

[79]	training's l2: 0.0421905	training's rmspe: 0.211397	valid_1's l2: 0.0425349	valid_1's rmspe: 0.212543
[80]	training's l2: 0.0416872	training's rmspe: 0.21014	valid_1's l2: 0.0420217	valid_1's rmspe: 0.211271
[81]	training's l2: 0.0415293	training's rmspe: 0.209661	valid_1's l2: 0.0418651	valid_1's rmspe: 0.210794
[82]	training's l2: 0.0411143	training's rmspe: 0.208554	valid_1's l2: 0.0414405	valid_1's rmspe: 0.209651
[83]	training's l2: 0.0407899	training's rmspe: 0.207671	valid_1's l2: 0.0411135	valid_1's rmspe: 0.20876
[84]	training's l2: 0.0403086	training's rmspe: 0.20637	valid_1's l2: 0.0406368	valid_1's rmspe: 0.207458
[85]	training's l2: 0.0399858	training's rmspe: 0.205446	valid_1's l2: 0.040318	valid_1's rmspe: 0.206546
[86]	training's l2: 0.0398461	training's rmspe: 0.205001	valid_1's l2: 0.0401789	valid_1's rmspe: 0.206096
[87]	training's l2: 0.0396513	training's rmspe: 0.204417	valid_1's l2: 0.0399844	valid_1's rmspe: 0.205519
[88]	training's l2: 0.0393838	training's 

[156]	training's l2: 0.0255433	training's rmspe: 0.163069	valid_1's l2: 0.0258766	valid_1's rmspe: 0.164342
[157]	training's l2: 0.0254465	training's rmspe: 0.162727	valid_1's l2: 0.0257774	valid_1's rmspe: 0.163995
[158]	training's l2: 0.0253882	training's rmspe: 0.162545	valid_1's l2: 0.0257197	valid_1's rmspe: 0.16381
[159]	training's l2: 0.025288	training's rmspe: 0.162222	valid_1's l2: 0.0256224	valid_1's rmspe: 0.163497
[160]	training's l2: 0.0251544	training's rmspe: 0.161769	valid_1's l2: 0.0254902	valid_1's rmspe: 0.163053
[161]	training's l2: 0.0250415	training's rmspe: 0.161405	valid_1's l2: 0.0253801	valid_1's rmspe: 0.1627
[162]	training's l2: 0.024987	training's rmspe: 0.161227	valid_1's l2: 0.0253225	valid_1's rmspe: 0.162513
[163]	training's l2: 0.0248934	training's rmspe: 0.160938	valid_1's l2: 0.0252282	valid_1's rmspe: 0.162224
[164]	training's l2: 0.0248158	training's rmspe: 0.160713	valid_1's l2: 0.0251553	valid_1's rmspe: 0.162014
[165]	training's l2: 0.0247096	tr

[234]	training's l2: 0.0193401	training's rmspe: 0.14177	valid_1's l2: 0.0196531	valid_1's rmspe: 0.143137
[235]	training's l2: 0.0193146	training's rmspe: 0.14167	valid_1's l2: 0.019629	valid_1's rmspe: 0.143046
[236]	training's l2: 0.0192607	training's rmspe: 0.141475	valid_1's l2: 0.0195752	valid_1's rmspe: 0.142849
[237]	training's l2: 0.0192095	training's rmspe: 0.141298	valid_1's l2: 0.019527	valid_1's rmspe: 0.142682
[238]	training's l2: 0.0191773	training's rmspe: 0.141164	valid_1's l2: 0.0194955	valid_1's rmspe: 0.142555
[239]	training's l2: 0.0190837	training's rmspe: 0.140831	valid_1's l2: 0.0194015	valid_1's rmspe: 0.142225
[240]	training's l2: 0.0190648	training's rmspe: 0.140756	valid_1's l2: 0.0193828	valid_1's rmspe: 0.142152
[241]	training's l2: 0.0190379	training's rmspe: 0.14065	valid_1's l2: 0.0193574	valid_1's rmspe: 0.142051
[242]	training's l2: 0.0190047	training's rmspe: 0.140523	valid_1's l2: 0.0193231	valid_1's rmspe: 0.141923
[243]	training's l2: 0.0189813	tr

...

[663]	training's l2: 0.0117256	training's rmspe: 0.110917	valid_1's l2: 0.0119653	valid_1's rmspe: 0.110783
[664]	training's l2: 0.0117169	training's rmspe: 0.110885	valid_1's l2: 0.0119556	valid_1's rmspe: 0.110749
[665]	training's l2: 0.0117112	training's rmspe: 0.110854	valid_1's l2: 0.0119502	valid_1's rmspe: 0.110721
[666]	training's l2: 0.0117048	training's rmspe: 0.110825	valid_1's l2: 0.0119439	valid_1's rmspe: 0.110692
[667]	training's l2: 0.0116975	training's rmspe: 0.11079	valid_1's l2: 0.0119372	valid_1's rmspe: 0.110661
[668]	training's l2: 0.0116908	training's rmspe: 0.110758	valid_1's l2: 0.0119304	valid_1's rmspe: 0.110625
[669]	training's l2: 0.0116871	training's rmspe: 0.11074	valid_1's l2: 0.0119268	valid_1's rmspe: 0.110608
[670]	training's l2: 0.0116761	training's rmspe: 0.110693	valid_1's l2: 0.0119166	valid_1's rmspe: 0.110562
[671]	training's l2: 0.0116735	training's rmspe: 0.11068	valid_1's l2: 0.0119149	valid_1's rmspe: 0.110553
[672]	training's l2: 0.0116666	

[741]	training's l2: 0.0111717	training's rmspe: 0.108339	valid_1's l2: 0.0114331	valid_1's rmspe: 0.108442
[742]	training's l2: 0.0111665	training's rmspe: 0.108314	valid_1's l2: 0.0114282	valid_1's rmspe: 0.108419
[743]	training's l2: 0.0111648	training's rmspe: 0.108306	valid_1's l2: 0.0114266	valid_1's rmspe: 0.108412
[744]	training's l2: 0.0111582	training's rmspe: 0.108275	valid_1's l2: 0.0114209	valid_1's rmspe: 0.108387
[745]	training's l2: 0.0111511	training's rmspe: 0.108239	valid_1's l2: 0.0114139	valid_1's rmspe: 0.108353
[746]	training's l2: 0.0111488	training's rmspe: 0.108228	valid_1's l2: 0.0114121	valid_1's rmspe: 0.108345
[747]	training's l2: 0.0111435	training's rmspe: 0.108201	valid_1's l2: 0.0114074	valid_1's rmspe: 0.108323
[748]	training's l2: 0.0111386	training's rmspe: 0.108172	valid_1's l2: 0.0114031	valid_1's rmspe: 0.108302
[749]	training's l2: 0.0111299	training's rmspe: 0.108126	valid_1's l2: 0.0113941	valid_1's rmspe: 0.108256
[750]	training's l2: 0.01112

[819]	training's l2: 0.0107141	training's rmspe: 0.10597	valid_1's l2: 0.0110046	valid_1's rmspe: 0.106317
[820]	training's l2: 0.0107087	training's rmspe: 0.105945	valid_1's l2: 0.0109995	valid_1's rmspe: 0.106293
[821]	training's l2: 0.0107022	training's rmspe: 0.105922	valid_1's l2: 0.0109941	valid_1's rmspe: 0.106274
[822]	training's l2: 0.0106974	training's rmspe: 0.105899	valid_1's l2: 0.0109899	valid_1's rmspe: 0.106255
[823]	training's l2: 0.0106905	training's rmspe: 0.105867	valid_1's l2: 0.0109832	valid_1's rmspe: 0.106225
[824]	training's l2: 0.010687	training's rmspe: 0.105851	valid_1's l2: 0.0109802	valid_1's rmspe: 0.106211
[825]	training's l2: 0.0106827	training's rmspe: 0.10583	valid_1's l2: 0.0109762	valid_1's rmspe: 0.106192
[826]	training's l2: 0.0106754	training's rmspe: 0.105795	valid_1's l2: 0.0109694	valid_1's rmspe: 0.10616
[827]	training's l2: 0.0106725	training's rmspe: 0.105775	valid_1's l2: 0.0109671	valid_1's rmspe: 0.106149
[828]	training's l2: 0.0106679	t

[897]	training's l2: 0.0103535	training's rmspe: 0.104109	valid_1's l2: 0.0106691	valid_1's rmspe: 0.104704
[898]	training's l2: 0.0103452	training's rmspe: 0.10407	valid_1's l2: 0.0106613	valid_1's rmspe: 0.104668
[899]	training's l2: 0.0103368	training's rmspe: 0.104029	valid_1's l2: 0.0106538	valid_1's rmspe: 0.104632
[900]	training's l2: 0.010329	training's rmspe: 0.103977	valid_1's l2: 0.0106467	valid_1's rmspe: 0.104597
[901]	training's l2: 0.0103239	training's rmspe: 0.103951	valid_1's l2: 0.0106419	valid_1's rmspe: 0.104573
[902]	training's l2: 0.0103193	training's rmspe: 0.103927	valid_1's l2: 0.010637	valid_1's rmspe: 0.104547
[903]	training's l2: 0.0103172	training's rmspe: 0.103916	valid_1's l2: 0.0106347	valid_1's rmspe: 0.104536
[904]	training's l2: 0.0103149	training's rmspe: 0.103904	valid_1's l2: 0.0106327	valid_1's rmspe: 0.104525
[905]	training's l2: 0.0103088	training's rmspe: 0.103877	valid_1's l2: 0.0106267	valid_1's rmspe: 0.1045
[906]	training's l2: 0.0103055	tr

[975]	training's l2: 0.0100107	training's rmspe: 0.102262	valid_1's l2: 0.0103519	valid_1's rmspe: 0.103133
[976]	training's l2: 0.0100088	training's rmspe: 0.102252	valid_1's l2: 0.0103498	valid_1's rmspe: 0.103122
[977]	training's l2: 0.0100072	training's rmspe: 0.102245	valid_1's l2: 0.0103487	valid_1's rmspe: 0.103116
[978]	training's l2: 0.0100021	training's rmspe: 0.102222	valid_1's l2: 0.0103437	valid_1's rmspe: 0.103094
[979]	training's l2: 0.00999945	training's rmspe: 0.102208	valid_1's l2: 0.0103411	valid_1's rmspe: 0.103082
[980]	training's l2: 0.00999585	training's rmspe: 0.102191	valid_1's l2: 0.0103377	valid_1's rmspe: 0.103067
[981]	training's l2: 0.00998876	training's rmspe: 0.102156	valid_1's l2: 0.0103302	valid_1's rmspe: 0.10303
[982]	training's l2: 0.00998595	training's rmspe: 0.102142	valid_1's l2: 0.0103278	valid_1's rmspe: 0.103018
[983]	training's l2: 0.00998375	training's rmspe: 0.10213	valid_1's l2: 0.0103256	valid_1's rmspe: 0.103007
[984]	training's l2: 0.00

In [55]:
np.expm1(pred)

array([ 4356.70337436,  7072.09832216,  9086.24014593, ...,
        6578.21208716, 23058.24325914,  6955.39102151])

In [56]:
print(rmspe(np.expm1(cv_train),np.expm1(y_train)))
pred[test_close_ind]=0
submission=pd.DataFrame({"Id":test_id,"Sales":np.round(np.expm1(pred),3)},columns=["Id","Sales"])

0.1860895071023409


In [57]:
from datetime import datetime
now=datetime.now()
submission.to_csv("data/{0:02d}{1:02d}{2:02d}{3:02d}_lgbm_submission.csv".format(now.year,now.month,now.day,now.hour),index=False)